In [1]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
import time

from ctmmodels.const import *

In [2]:
currdir = os.getcwd()
regex_name = r'(([A-Z])\w+)'

In [3]:
# CHANGE THIS FOR EACH PROCESSING

IMAGE_PATH = 'graphs/experiments-3'
DF_PATH = 'experiments-final3/'

In [4]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

In [5]:
_approach = SOUTHBOUND

_approach_terms = [
    'Left',
    'Through',
    'Right'
]

_cell_path = [
    (CELL_SOURCE,0,_approach),
    (CELL_NORMAL,0,_approach),
    (CELL_NORMAL,1,_approach),
    (CELL_NORMAL,2,_approach),
    (CELL_MOVEMENT,THROUGH_TURN,_approach),
    S_mapping((CELL_MOVEMENT,THROUGH_TURN,_approach))[0]
]

_movement_labels = {
    (2,LEFT_TURN,NORTHBOUND): 'NBL',
    (2,RIGHT_TURN,NORTHBOUND): 'NBR',
    (2,THROUGH_TURN,NORTHBOUND): 'NBT',
    (2,LEFT_TURN,SOUTHBOUND): 'SBL',
    (2,RIGHT_TURN,SOUTHBOUND): 'SBR',
    (2,THROUGH_TURN,SOUTHBOUND): 'SBT',
    (2,LEFT_TURN,EASTBOUND): 'EBL',
    (2,RIGHT_TURN,EASTBOUND): 'EBR',
    (2,THROUGH_TURN,EASTBOUND): 'EBT',
    (2,LEFT_TURN,WESTBOUND): 'WBL',
    (2,RIGHT_TURN,WESTBOUND): 'WBR',
    (2,THROUGH_TURN,WESTBOUND): 'WBT',
}

_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

_col_rename = {
    'Runtime': 'runtime',
    'Delay': 'delay',
    'Throughput': 'throughput',
    'ObjValue': 'objective_value'
}

def movement_paths(approach):
    return [
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,LEFT_TURN,approach),
            S_mapping((CELL_MOVEMENT,LEFT_TURN,approach))[0]
        ],
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,THROUGH_TURN,approach),
            S_mapping((CELL_MOVEMENT,THROUGH_TURN,approach))[0]
        ],
        [
            (CELL_SOURCE,0,approach),
            (CELL_NORMAL,0,approach),
            (CELL_NORMAL,1,approach),
            (CELL_NORMAL,2,approach),
            (CELL_MOVEMENT,RIGHT_TURN,approach),
            S_mapping((CELL_MOVEMENT,RIGHT_TURN,approach))[0]
        ]
    ]

# Experiments Results Processing

## Obtaining list of columns

In [6]:
filename = currdir + '/{}sim-results/result_d450_a0.5_b0.5_c0.xls'.format(DF_PATH)
df_tmp = pd.read_excel(filename, sheet_name='Speed').fillna(0)

In [7]:
df_tmp.head()

,t,0x20 - approach2_lane3,0x7 - approach1_lane2,0x25 - approach1_lane1,0x16 - exit0_lane3,0x32 - approach0_lane3,0x2 - approach0_lane2,0x28 - approach1_lane3,0x17 - exit0_lane1,0x1d - approach2_lane1,...,0x29 - approach1_lane3,0x10 - exit3_lane3,0x12 - exit2_lane3,0x13 - exit2_lane1,0x14 - exit1_lane3,0x2e - approach0_lane1,0x18 - approach3_lane3,0xc - approach2_lane2,0xb - exit2_lane2,0x2a - approach1_lane3
0,60,77.760000,10.407273,0.000000,82.800000,76.406400,33.145263,0.0000,0.000000,29.275826,...,92.091429,116.640000,90.720000,0.000000,87.892364,46.505354,0.0000,9.521679,66.461538,76.924800
1,120,77.760000,8.427975,16.414476,82.800000,70.592870,32.919273,0.0000,55.296000,8.695322,...,89.949767,88.868571,90.720000,61.560000,84.808421,42.714157,0.0000,4.958310,58.529032,76.924800
2,180,77.760000,8.342672,9.604755,83.671579,72.276324,24.490667,0.0000,55.296000,4.764061,...,87.162592,85.945263,71.950345,53.110588,80.568000,43.111087,77.5584,4.515267,60.480000,75.975111
3,240,77.760000,7.223852,7.345705,89.081379,70.528985,11.629477,76.9248,56.552727,3.858202,...,83.110612,89.341277,72.110769,51.984000,84.404571,42.891762,86.1760,4.835312,58.800000,71.944058
4,300,80.441379,7.316391,6.403170,86.286316,71.312640,9.436143,76.9248,58.206316,3.381750,...,83.972842,91.966154,72.576000,54.052683,84.254118,41.628203,86.1760,4.644216,57.906383,72.117000


In [8]:
# Filter out some of the rows

delay_rows = ["{} Delay".format(d) for d in range(60,601,60)]
delay_rename = dict([
    ("{} Delay".format(d), d)
    for d in range(60,601,60)
])

thru_rows = ["{} Throughput".format(d) for d in range(60,601,60)]
thru_rename = dict([
    ("{} Throughput".format(d), d)
    for d in range(60,601,60)
])

vol_rows = [d for d in range(60,601,60)]

In [9]:
# Filter out some of the columns

survey_zones = df_tmp.columns.values[1:]

entrances = [x for x in survey_zones if 'approach' in x]
exits = [y for y in survey_zones if 'exit' in y]

print("{} approach zones, {} exit zones".format(len(entrances), len(exits)))

48 approach zones, 12 exit zones


In [10]:
reg_approach = re.compile('0x(\w+) - approach.*')
reg_exit = re.compile('0x(\w+) - exit.*')

entrances_rename = dict([
    (k, SURVEY_ZONE_MAPPING[reg_approach.match(k).group(1)])
    for k in entrances
])

exits_rename = dict([
    (k, SURVEY_ZONE_MAPPING[reg_exit.match(k).group(1)])
    for k in exits
])

## Preprocessing the results

In [11]:
weightlist = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [12]:
total_thru_dfs = []
total_delay_dfs = []
ave_delay_dfs = []

for demand in demands:
    for weights in weightlist:

        partial_fn = 'old' if weights == (0, 0, 0) else 'a{}_b{}_c{}'.format(*weights)
        filename = currdir + '/{}sim-results/result_d{}_{}.xls'.format(DF_PATH, demand, partial_fn)
        df_tmp = pd.read_excel(filename, sheet_name='Speed').fillna(0)

        # Get the cumulative delays

        df_tmp1 = df_tmp[df_tmp.t.isin(delay_rows)]
        df_tmp1['t'].replace(delay_rename, inplace=True)
        df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
        df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
        df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

        df_cumdelay = df_tmp1.pivot(index='t', columns='level_1', values=0)
        df_cumdelay.to_pickle(currdir + '/{}cumu-delay/delay_d{}_a{}_b{}_c{}.pkl'.format(DF_PATH, demand, *weights))
        
        # Get the cumulative throughputs

        df_tmp3 = df_tmp[df_tmp.t.isin(thru_rows)]
#         if weights == (0, 0, 0):
#             for col in ['0x11 - exit3_lane1', '0x15 - exit1_lane1', '0x17 - exit0_lane1', '0x13 - exit2_lane1']:
#                 if col not in df_tmp3.columns:
#                     df_tmp3[col] = 0
        df_tmp3['t'].replace(thru_rename, inplace=True)
        df_tmp3 = df_tmp3.sort_values(by='t').set_index('t')[exits].stack().reset_index()
        df_tmp3['level_1'] = df_tmp3['level_1'].map(exits_rename)
        df_tmp3 = df_tmp3.groupby(['t', 'level_1']).sum().reset_index()

        df_cumthru = df_tmp3.pivot(index='t', columns='level_1', values=0)
        df_cumthru.to_pickle(currdir + '/{}cumu-thru/thru_d{}_a{}_b{}_c{}.pkl'.format(DF_PATH, demand, *weights))
        
        # Get volumes
        
        df_tmp = pd.read_excel(filename, sheet_name='Volume').fillna(0)
        df_tmp1 = df_tmp[df_tmp.t.isin(vol_rows)]
        df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
        df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
        df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

        df_volume = df_tmp1.pivot(index='t', columns='level_1', values=0)
        df_volume.to_pickle(currdir + '/{}actual-volume/vol_d{}_a{}_b{}_c{}.pkl'.format(DF_PATH, demand, *weights))

        # Get aggregate values

        df_total_thru_tmp = df_cumthru.loc[600, :].reset_index().set_index('level_1').T
        df_total_thru_tmp['demand'] = [demand]
        df_total_thru_tmp['new_model'] = weights != (0, 0, 0)
        df_total_thru_tmp['alpha'] = weights[0]
        df_total_thru_tmp['beta'] = weights[1]
        df_total_thru_tmp['gamma'] = weights[2]
        df_total_thru_tmp['model_type'] = _model_type[weights]
        total_thru_dfs.append(df_total_thru_tmp)

        df_total_delay_tmp = df_cumdelay.loc[600, :].reset_index().set_index('level_1').T
        df_total_delay_tmp['demand'] = [demand]
        df_total_delay_tmp['new_model'] = weights != (0, 0, 0)
        df_total_delay_tmp['alpha'] = weights[0]
        df_total_delay_tmp['beta'] = weights[1]
        df_total_delay_tmp['gamma'] = weights[2]
        df_total_delay_tmp['model_type'] = _model_type[weights]
        total_delay_dfs.append(df_total_delay_tmp)

        df_tmp2 = df_tmp[df_tmp.t == 'Average Delay'][entrances].rename(columns=entrances_rename).stack().reset_index()
        df_ave_delay_tmp = df_tmp2.groupby('level_1').mean().reset_index().pivot(index='level_0', columns='level_1', values=0)
        df_ave_delay_tmp['demand'] = [demand]
        df_ave_delay_tmp['new_model'] = weights != (0, 0, 0)
        df_ave_delay_tmp['alpha'] = weights[0]
        df_ave_delay_tmp['beta'] = weights[1]
        df_ave_delay_tmp['gamma'] = weights[2]
        df_ave_delay_tmp['model_type'] = _model_type[weights]
        ave_delay_dfs.append(df_ave_delay_tmp)

/home/localsim/.virtualenvs/thesis/local/lib/python2.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [13]:
df_total_thru = pd.concat(total_thru_dfs).reset_index().iloc[:, 1:]
df_total_thru.to_pickle(currdir + '/{}total_throughput.pkl'.format(DF_PATH))

df_total_delay = pd.concat(total_delay_dfs).reset_index().iloc[:, 1:]
df_total_delay.to_pickle(currdir + '/{}total_delay.pkl'.format(DF_PATH))

df_ave_delay = pd.concat(ave_delay_dfs).reset_index().iloc[:, 1:]
df_ave_delay.to_pickle(currdir + '/{}ave_delay.pkl'.format(DF_PATH))

# Greentimes from Experiment

## Setting variables for image generation

## Comparing greentimes

In [14]:
def plot_greentime_map(dfg, title="Greentime maps", headless=False, filename='greentime.png', movements=True):
    fig, axs = plt.subplots(figsize=(10,5))

    dfg_matrix = dfg.T
    if movements:
        dfg_matrix = dfg_matrix.reindex(index=[
            (2,LEFT_TURN,NORTHBOUND), (2,RIGHT_TURN,NORTHBOUND), (2,THROUGH_TURN,NORTHBOUND),
            (2,LEFT_TURN,SOUTHBOUND), (2,RIGHT_TURN,SOUTHBOUND), (2,THROUGH_TURN,SOUTHBOUND),
            (2,LEFT_TURN,EASTBOUND), (2,RIGHT_TURN,EASTBOUND), (2,THROUGH_TURN,EASTBOUND),
            (2,LEFT_TURN,WESTBOUND), (2,RIGHT_TURN,WESTBOUND), (2,THROUGH_TURN,WESTBOUND),
        ]).rename(index=_movement_labels).rename_axis(index='Movements', columns='Timesteps')
    sns.heatmap(data=dfg_matrix, cbar=False)
    
    fig.suptitle(title, fontsize=18)
    
    if headless:
        fig.savefig(IMAGE_PATH + filename)
        plt.close(fig)

## Saving Dataframes

In [15]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

## Image generation

In [16]:
_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

for demand in _demands:
    for weights in _weights:
        if weights == (0,0,0):
            df_e0 = pd.read_pickle(DF_PATH + 'greentimes/initial/greentimes_d{}_old.pkl'.format(demand))
            df_e1 = pd.read_pickle(DF_PATH + 'greentimes/greentimes_d{}_epoch1_old.pkl'.format(demand))
            df_e2 = pd.read_pickle(DF_PATH + 'greentimes/greentimes_d{}_epoch2_old.pkl'.format(demand))
            df_e3 = pd.read_pickle(DF_PATH + 'greentimes/greentimes_d{}_epoch3_old.pkl'.format(demand))
            subtitle = 'Old model'
            filename = 'old'
        else:
            df_e0 = pd.read_pickle(DF_PATH + 'greentimes/initial/greentimes_d{}_new_a{}_b{}_c{}.pkl'.format(demand, *weights))
            df_e1 = pd.read_pickle(DF_PATH + 'greentimes/greentimes_d{}_epoch1_a{}_b{}_c{}.pkl'.format(demand, *weights))
            df_e2 = pd.read_pickle(DF_PATH + 'greentimes/greentimes_d{}_epoch2_a{}_b{}_c{}.pkl'.format(demand, *weights))
            df_e3 = pd.read_pickle(DF_PATH + 'greentimes/greentimes_d{}_epoch3_a{}_b{}_c{}.pkl'.format(demand, *weights))
            subtitle = 'New model ({})'.format(_model_type[weights])
            filename = 'a{}_b{}_c{}'.format(*weights)
            
        plot_greentime_map(
            df_e0,
            title='Greentimes for demand {} and {} (Initial)'.format(demand, subtitle),
            headless=True,
            filename='d{}_{}_epoch0.png'.format(demand, filename)
        )

        plot_greentime_map(
            df_e1,
            title='Greentimes for demand {} and {} (Epoch 1)'.format(demand, subtitle),
            headless=True,
            filename='d{}_{}_epoch1.png'.format(demand, filename)
        )
        
        plot_greentime_map(
            df_e2,
            title='Greentimes for demand {} and {} (Epoch 2)'.format(demand, subtitle),
            headless=True,
            filename='d{}_{}_epoch2.png'.format(demand, filename)
        )
        
        plot_greentime_map(
            df_e3,
            title='Greentimes for demand {} and {} (Epoch 3)'.format(demand, subtitle),
            headless=True,
            filename='d{}_{}_epoch3.png'.format(demand, filename)
        )

# Results from MILP Realtime Computations

## Combining across epochs

In [17]:
_results_dflist = []

for demand in _demands:
    for weights in _weights:
        if weights == (0,0,0):
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_old.csv'.format(demand))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_old.csv'.format(demand))
            df_e3 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch3_old.csv'.format(demand))
        else:
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_a{}_b{}_c{}.csv'.format(demand, *weights))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_a{}_b{}_c{}.csv'.format(demand, *weights))
            df_e3 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch3_a{}_b{}_c{}.csv'.format(demand, *weights))

        df_e1 = df_e1.rename(columns=_col_rename)
        df_e1['demand'] = [demand]
        df_e1['new_model'] = weights != (0,0,0)
        df_e1['alpha'] = weights[0]
        df_e1['beta'] = weights[1]
        df_e1['gamma'] = weights[2]
        df_e1['model_type'] = _model_type[weights]
        df_e1['epoch'] = 1
        
        df_e2 = df_e2.rename(columns=_col_rename)
        df_e2['demand'] = [demand]
        df_e2['new_model'] = weights != (0,0,0)
        df_e2['alpha'] = weights[0]
        df_e2['beta'] = weights[1]
        df_e2['gamma'] = weights[2]
        df_e2['model_type'] = _model_type[weights]
        df_e2['epoch'] = 2
        
        df_e3 = df_e3.rename(columns=_col_rename)
        df_e3['demand'] = [demand]
        df_e3['new_model'] = weights != (0,0,0)
        df_e3['alpha'] = weights[0]
        df_e3['beta'] = weights[1]
        df_e3['gamma'] = weights[2]
        df_e3['model_type'] = _model_type[weights]
        df_e3['epoch'] = 3
        
        _results_dflist.append(pd.concat([df_e1, df_e2, df_e3]))

In [18]:
df = pd.concat(_results_dflist)

In [19]:
df_initial_results = pd.read_pickle(DF_PATH + 'results.pkl')

In [20]:
df_initial_results['demand'] = df_initial_results.apply(lambda row: 
                                                        int(row['demand_ns']) if row['demand_ns'] == row['demand_ew']
                                                        else (int(row['demand_ns']), int(row['demand_ew']))
                                                        , axis=1)

df_initial_results['new_model'] = df_initial_results.apply(lambda row: row['alpha'] != 0, axis=1)

df_initial_results['model_type'] = df_initial_results.apply(lambda row: _model_type[
    (row['alpha'], row['beta'], row['gamma'])
], axis=1)

df_initial_results['epoch'] = 0

df_initial_results['delay'] = df_initial_results['delay'].apply(lambda x: 2*x)

df_initial_results = df_initial_results[[
    'runtime',
    'delay',
    'throughput',
    'objective_value',
    'demand',
    'new_model',
    'alpha',
    'beta',
    'gamma',
    'model_type',
    'epoch'
]]

df = df.rename(columns={'obj-value': 'objective_value'})

In [21]:
df_final = pd.concat([df_initial_results, df]).sort_values(by=['epoch', 'demand', 'model_type'])

save_df(df_final, 'milp-realtime-results')

# Expected vs Actual Results

In [22]:
delay_dfs = []
throughput_dfs = []

df_milp = pd.read_pickle(DF_PATH + 'milp-realtime-results.pkl')
df_delay = pd.DataFrame(columns=['demand', 'model_type', 'expected', 'actual', 'epoch'])
df_thru = pd.DataFrame(columns=['demand', 'model_type', 'expected', 'actual', 'epoch'])

for demand in _demands:
    for weights in _weights:

        # Get actual values
        
        df_cumdelay_tmp = pd.read_pickle(DF_PATH + 'cumu-delay/delay_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_instdelay_tmp = df_cumdelay_tmp.diff()
        df_instdelay_tmp.loc[60, :] = df_cumdelay_tmp.loc[60, :]
        actual_delay = list(df_instdelay_tmp.sum(axis=1)[[60, 240, 420]])
        
        df_cumthru_tmp = pd.read_pickle(DF_PATH + 'cumu-thru/thru_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        df_instthru_tmp = df_cumthru_tmp.diff()
        df_instthru_tmp.loc[60, :] = df_cumthru_tmp.loc[60, :]
        actual_thru = list(df_instthru_tmp.sum(axis=1)[[60, 240, 420]])

        # Get expected values
        
        expected_delay = list(df_milp[(df_milp.demand == demand) & (df_milp.model_type == _model_type[weights])].delay)
        expected_thru = list(df_milp[(df_milp.demand == demand) & (df_milp.model_type == _model_type[weights])].throughput)
        
        # Combine into a dataframe
        
        for i in range(3):
            df_delay.loc[-1] = [demand, _model_type[weights], expected_delay[i], actual_delay[i], i]
            df_thru.loc[-1] = [demand, _model_type[weights], expected_thru[i], actual_thru[i], i]
            df_delay.index += 1
            df_thru.index += 1

In [23]:
df_delay.to_pickle(DF_PATH + "delay_comparison.pkl")
df_thru.to_pickle(DF_PATH + "throughput_comparison.pkl")